# Criação de um Dataframe de vendas

## 1. Importações de bibliotecas

In [ ]:

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import os

print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


## 2. Configuração inicial

In [ ]:

np.random.seed(46)
random.seed(46)

print("Sementes definidas para reprodutibilidade.")


Sementes definidas para reprodutibilidade.


In [ ]:

output_dir = "datasets"
os.makedirs(output_dir, exist_ok=True)

print(f"Pasta '{output_dir}' criada (ou já existia).")


Pasta 'datasets' criada (ou já existia).


## Definição dos parâmetros e lista de variaveis

In [ ]:

num_registros = 3000

print(f"Criando DataFrame com {num_registros} registros...")


'Define quantos registros (linhas) o DataFrame terá inicialmente.'

In [ ]:
datas = pd.date_range(start="2023-01-01", end="2023-12-31", freq="D")

print(f"Datas geradas de {datas.min().date()} a {datas.max().date()}.")


'Gera uma sequência diária de datas entre 01/01/2023 e 31/12/2023 (inclusive).\n\ndatas é um DatetimeIndex com todos os dias do ano (365 entradas).'

In [ ]:
produtos = ["Notebook", "Smartphone", "Mouse", "Teclado", "Monitor", "Fone de Ouvido", "Impressora", "Cadeira Gamer"]
categorias = ["Informática", "Acessórios", "Periféricos", "Móveis de Escritório"]
pagamentos = ["Crédito", "Débito", "Pix", "Dinheiro", "Boleto"]

print(f"Produtos, categorias e métodos de pagamento definidos.")

'Listas de valores possíveis para as colunas categóricas — usadas para amostragem aleatória.'

## 4. Simulação do Dataset

In [ ]:
'''"ID": range(1, num_registros + 1) → cria IDs sequenciais (1..1000).

"Data": np.random.choice(datas, size=num_registros) → escolhe aleatoriamente uma data (do DatetimeIndex) para cada linha. Resultado: coluna de tipo datetime (na maioria dos casos ficará como datetime64[ns] até que se insiram NaN).

"Produto", "Categoria", "Pagamento" usam np.random.choice para escolher categorias aleatórias.

"Quantidade": np.random.randint(1, 10, size=num_registros) → inteiros aleatórios entre 1 e 9 (inclusive o 1, exclusivo 10).

"Preço": np.round(np.random.uniform(50, 5000, size=num_registros), 2) → preços float entre 50 e 5000, arredondados para 2 casas (simula ticket/valor unitário).
Observações de tipo:

Antes de inserir NaN, ID é int, Quantidade int, Preço float, Data datetime, outras object (strings).'''

df = pd.DataFrame({
    "ID": range(1, num_registros + 1),
    "Data": np.random.choice(datas, size=num_registros),
    "Produto": np.random.choice(produtos, size=num_registros),
    "Categoria": np.random.choice(categorias, size=num_registros),
    "Quantidade": np.random.randint(1, 10, size=num_registros),
    "Preço": np.round(np.random.uniform(50, 5000, size=num_registros), 2),
    "Pagamento": np.random.choice(pagamentos, size=num_registros)
})

print("DataFrame criado com sucesso.")

'"ID": range(1, num_registros + 1) → cria IDs sequenciais (1..1000).\n\n"Data": np.random.choice(datas, size=num_registros) → escolhe aleatoriamente uma data (do DatetimeIndex) para cada linha. Resultado: coluna de tipo datetime (na maioria dos casos ficará como datetime64[ns] até que se insiram NaN).\n\n"Produto", "Categoria", "Pagamento" usam np.random.choice para escolher categorias aleatórias.\n\n"Quantidade": np.random.randint(1, 10, size=num_registros) → inteiros aleatórios entre 1 e 9 (inclusive o 1, exclusivo 10).\n\n"Preço": np.round(np.random.uniform(50, 5000, size=num_registros), 2) → preços float entre 50 e 5000, arredondados para 2 casas (simula ticket/valor unitário).\nObservações de tipo:\n\nAntes de inserir NaN, ID é int, Quantidade int, Preço float, Data datetime, outras object (strings).'

## 5. Criação e Inserção de valores nulos

In [ ]:
'''df.size = número total de células do DataFrame = n_linhas * n_colunas (1000 * 7 = 7000).

n_nulos = 5% de 7000 = 350 (no exemplo com 1000 linhas). Ou seja, serão transformadas em NaN 350 células aleatória'''

nulos_porcentagem = 0.05
cols_sem_id = df.columns[df.columns != "ID"]  # todas as colunas exceto ID
n_celulas = df[cols_sem_id].size
n_nulos = int(n_celulas * nulos_porcentagem)

print(f"Inserindo {n_nulos} valores NaN aleatórios ({nulos_porcentagem*100}% das células, exceto ID)...")

'df.size = número total de células do DataFrame = n_linhas * n_colunas (1000 * 7 = 7000).\n\nn_nulos = 5% de 7000 = 350 (no exemplo com 1000 linhas). Ou seja, serão transformadas em NaN 350 células aleatória'

In [ ]:
'''Para cada uma das 350 iterações:

i é um índice de linha aleatório (0..999).

j é um índice de coluna aleatório (0..6).

df.iat[i, j] = np.nan escreve NaN naquela célula (indexação por posição inteira — muito rápida para leituras/escritas escalares).
Efeitos importantes

Pode atingir qualquer coluna, inclusive ID ou Data. Se NaN for inserido numa coluna de inteiros (ID ou Quantidade), o pandas converte automaticamente essa coluna para float (porque NaN é um float especial em pandas). Isso é esperado e um ponto essencial a se lembrar quando tratar tipos depois.

Inserir NaN na coluna Data fará com que ela deixe de ser datetime64 e passe a ter tipo object ou float (dependendo de quantos NaN), exigindo pd.to_datetime() depois para reconverter.

df.iat usa posições inteiras: é diferente de .loc (rótulo) e .iloc (posição por fatia). .iat é o mais rápido para acessar um único elemento.

Observação de performance

Para 1000 linhas, este loop é OK. Para milhões de linhas, é preferível usar estratégias vetorizadas (por ex. amostrar pares (row,col) sem loop).'''

for _ in range(n_nulos):
    i = np.random.randint(0, df.shape[0])         # índice da linha
    j = np.random.randint(0, len(cols_sem_id))    # índice da coluna (sem ID)
    col = cols_sem_id[j]
    df.at[i, col] = np.nan

print("Valores NaN inseridos.")
   

## 6. Gerando e Inserindo registros duplicados

In [38]:
'''n_duplicatas = 3% de 1000 = 30 (aprox.).

df.sample(...) seleciona aleatoriamente 30 linhas do DataFrame atual (note que isto é feito após inserir os NaN, então as linhas amostradas podem incluir NaN nas mesmas posições).

pd.concat([df, duplicatas], ignore_index=True) concatena o DataFrame original com as linhas amostradas. ignore_index=True reindexa o DataFrame concatenado com novos índices 0..N-1.
Por que isso mantém o mesmo ID

duplicatas é uma cópia exata das linhas selecionadas — incluindo o valor de ID. Ao concatenar, essas linhas duplicadas carregam o mesmo valor de ID da linha original. Ou seja, você terá IDs repetidos — exatamente o que você pediu para simular duplicação de dados de sistema.
Observações

Atenção: se você tivesse reatribuído IDs depois (como em uma versão anterior do script), as duplicatas não seriam reconhecidas por df.duplicated() porque o ID na linha duplicada seria diferente. Aqui, mantemos IDs iguais de propósito.'''

n_duplicatas = int(0.03 * len(df))
duplicatas = df.sample(n=n_duplicatas, random_state=42)
df = pd.concat([df, duplicatas], ignore_index=True)

print(f"{n_duplicatas} linhas duplicadas inseridas. DataFrame agora tem {len(df)} linhas.")

92 linhas duplicadas inseridas. DataFrame agora tem 3182 linhas.


## 7. Salvando o Dataset 

In [ ]:
'''os.path.join monta o caminho correto para o arquivo (ajuda em compatibilidade entre sistemas).

df.to_csv(..., index=False) salva o DataFrame em CSV sem a coluna de índice do pandas (apenas as colunas reais).'''

output_path = os.path.join(output_dir, "vendas_raw.csv")
df.to_csv(output_path, index=False)

print(f"DataFrame salvo em '{output_path}'.")

'os.path.join monta o caminho correto para o arquivo (ajuda em compatibilidade entre sistemas).\n\ndf.to_csv(..., index=False) salva o DataFrame em CSV sem a coluna de índice do pandas (apenas as colunas reais).'

## 8. Prints para verificação do Dataset criado

In [30]:

print(f"Total de registros: {len(df)}")


Total de registros: 3090


In [31]:

print(f"Duplicatas: {df.duplicated().sum()}")

Duplicatas: 90


In [32]:

print(f"Nulos totais: {df.isnull().sum().sum()}")

Nulos totais: 904


In [33]:

print("\nResumo por coluna:")
print(df.info())


Resumo por coluna:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3090 entries, 0 to 3089
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          3090 non-null   int64         
 1   Data        2936 non-null   datetime64[ns]
 2   Produto     2942 non-null   object        
 3   Categoria   2916 non-null   object        
 4   Quantidade  2954 non-null   float64       
 5   Preço       2934 non-null   float64       
 6   Pagamento   2954 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 169.1+ KB
None
